---

# Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install implicit==0.6

  Using cached implicit-0.6.0-cp310-cp310-manylinux2014_x86_64.whl (18.6 MB)


In [1]:
import pandas as pd  # Импорт библиотеки pandas для работы с данными в виде таблиц
import numpy as np  # Импорт библиотеки numpy для работы с массивами и математическими операциями
import matplotlib.pyplot as plt  # Импорт библиотеки для построения графиков
%matplotlib inline  # Магическая команда для отображения графиков в Jupyter Notebook
%config InlineBackend.figure_format = 'retina'  # Установка формата отображения графиков для улучшения качества на экранах с высоким разрешением

from scipy.sparse import csr_matrix  # Импорт функции csr_matrix из библиотеки scipy.sparse для работы с разреженными матрицами

from implicit.bpr import BayesianPersonalizedRanking  # Импорт модуля для использования метода Bayesian Personalized Ranking

# Матричная факторизация
from implicit.als import AlternatingLeastSquares  # Импорт метода Alternating Least Squares для матричной факторизации
from implicit.nearest_neighbours import ItemItemRecommender  # Импорт рекомендателя Item-Item
from implicit.nearest_neighbours import bm25_weight, tfidf_weight  # Импорт методов для взвешивания признаков (BM25, TF-IDF)

# Функции из 1-ого вебинара
import os, sys  # Импорт модулей для работы с операционной системой и системными параметрами
import inspect  # Импорт модуля для получения информации об объектах во время выполнения
import shutil  # Импорт модуля для работы с файлами и папками

module_path = os.path.abspath(os.path.join(os.pardir))  # Получение абсолютного пути к родительской директории текущего модуля
if module_path not in sys.path:  # Проверка наличия пути в системной переменной sys.path
    sys.path.append(module_path)  # Добавление пути в системную переменную sys.path

# Импорт пользовательских функций и классов
from src.metrics import precision_at_k, recall_at_k  # Импорт метрик precision_at_k и recall_at_k из модуля src.metrics
from src.utils import prefilter_items  # Импорт функции prefilter_items из модуля src.utils
from src.recommenders import MainRecommender  # Импорт класса MainRecommender из модуля src.recommenders




In [2]:
import sys
print("User Current Version:\n-",sys.version)

User Current Version:
- 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [2]:
# Перенесём функции метрик в модуль src.metrics.py
import os

# Название директории и файл
directory_name = 'src'
file_name = 'metrics.py'

# Проверяем существование директории, и если её нет, создаем
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Код класса, который нужно поместить в файл
func_code = '''

import pandas as pd
import numpy as np


# функции являющиеся метриками:
#hit_rate: вычисляет показатель попадания.
#Возвращает 1, если хотя бы один элемент из рекомендованного списка присутствует в списке покупок, и 0 в противном случае.
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1


#hit_rate_at_k: вычисляет показатель попадания на первых k позициях.
#Использует функцию hit_rate для оценки только первых k элементов из рекомендованного списка.
def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)


#precision: вычисляет точность.
#Определяет долю элементов из рекомендованного списка, которые присутствуют в списке покупок.
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


#precision_at_k: вычисляет точность на первых k позициях.
#Использует функцию precision для оценки только первых k элементов из рекомендованного списка.
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)


#money_precision_at_k: вычисляет точность с учетом стоимости элементов.
#Умножает бинарные флаги на цены рекомендованных элементов и делит сумму на общую стоимость рекомендаций.
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_recommended.sum()


#recall: вычисляет полноту.
# Определяет долю элементов из списка покупок, которые присутствуют в рекомендованном списке.
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


#recall_at_k: вычисляет полноту на первых k позициях.
# Использует функцию recall для оценки только первых k элементов из рекомендованного списка.
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


#money_recall_at_k: вычисляет полноту с учетом стоимости элементов.
#Умножает бинарные флаги на цены рекомендованных элементов и делит сумму на общую стоимость покупок.
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()


#ap_k: вычисляет среднюю точность на первых k позициях.
#Вычисляет точность на каждой позиции из рекомендованного списка до k и возвращает среднее значение.
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[recommended_list <= k]

    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    amount_relevant = len(relevant_indexes)

    sum_ = sum(
        [precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    return sum_ / amount_relevant

'''

# Путь к файлу .py
file_path = os.path.join(directory_name, file_name)

# Создаем файл и записываем код в него
with open(file_path, 'w') as file:
    file.write(func_code)

In [3]:


def prefilter_items(df, item_features=None, take_n_popular=5000):
    # Уберем самые популярные товары (их и так купят)
    popularity = df.groupby('item_id')['user_id'].nunique().reset_index()
    popularity['share_unique_users'] = popularity['user_id'] / df['user_id'].nunique()
    top_popular = popularity[popularity['share_unique_users'] > 0.5]['item_id'].tolist()
    df = df[~df['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.0025].item_id.tolist()
    df = df[~df['item_id'].isin(top_notpopular)]

    # Уберем товары, которые не продавались за последние 12 месяцев
    recently_sold = df[df['day'] < 386]['item_id'].tolist()
    df = df[df['item_id'].isin(recently_sold)]

    # Уберем не интересные для рекоммендаций категории (department)
    unique_departments = item_features.groupby('department')['item_id'].nunique()
    top_departments = unique_departments[unique_departments > 100].index.tolist()
    top_items_deps = item_features[item_features['department'].isin(top_departments)]
    df = df[df['item_id'].isin(top_items_deps['item_id'].unique())]

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    price_weights = df.groupby('item_id')['sales_value'].min() / df['sales_value'].max()
    least_expensive = price_weights[price_weights < 0.0005].index.tolist()
    df = df[~df['item_id'].isin(least_expensive)]

    # Уберем слишком дорогие товары
    top_expensive = price_weights[price_weights > 0.12].index.tolist()
    df = df[~df['item_id'].isin(top_expensive)]

    # Возьмем 5000 раиболее полулярных товаров
    popular_5000 = df.groupby('item_id')['quantity'].count().reset_index()
    top_5000 = popular_5000.sort_values(by='quantity', ascending=False)['item_id'].head(take_n_popular).tolist()
    df.loc[~df['item_id'].isin(top_5000), 'item_id'] = 999999

    # Возвращаем данные
    return df



# Создание папки src
src_dir = 'src'
os.makedirs(src_dir, exist_ok=True)

# Список функций для переноса
functions_to_move = [
    prefilter_items
]

# Перенос функций в src.metrics.py
with open(os.path.join(src_dir, 'utils.py'), 'a', encoding='utf-8') as f:
    for function in functions_to_move:
        function_source = inspect.getsource(function)
        f.write(function_source)
        f.write('\n')

In [3]:
# Чтение данных из CSV-файла
data = pd.read_csv('/content/drive/MyDrive/lesson_4/lesson_4/homework/retail_train.csv')

# Приведение названий столбцов к нижнему регистру
data.columns = [col.lower() for col in data.columns]

# Переименование столбцов для удобства работы с данными
data.rename(columns={'household_key': 'user_id', 'product_id': 'item_id'}, inplace=True)

# Задание размера тестового набора в неделях
test_size_weeks = 3

# Создание обучающего набора данных
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]

# Создание тестового набора данных
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Вывод первых 5 строк обучающего набора данных
data_train.head(5)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
# Чтение данных из CSV-файла о продуктах
item_features = pd.read_csv('/content/drive/MyDrive/lesson_4/lesson_4/homework/product.csv')

# Приведение названий столбцов к нижнему регистру для удобства работы
item_features.columns = [col.lower() for col in item_features.columns]

# Переименование столбца product_id на item_id для единообразия идентификаторов товаров
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

# Вывод первых 5 строк данных о продуктах для предварительного анализа
item_features.head(5)


,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
# Группировка данных тестового набора по 'user_id' и создание массива уникальных 'item_id' для каждого пользователя
result = data_test.groupby('user_id')['item_id'].unique().reset_index()

# Переименование столбцов 'user_id' и 'actual' в полученном DataFrame
result.columns = ['user_id', 'actual']

# Вывод первых 5 строк полученного DataFrame
result.head(5)


,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [7]:
# Получение количества уникальных товаров до предварительной фильтрации
n_items_before = data_train['item_id'].nunique()

# Применение предварительной фильтрации товаров с помощью функции prefilter_items
# take_n_popular - количество самых популярных товаров, которые будут оставлены
data_train = prefilter_items(data_train, item_features, take_n_popular=5000)

# Получение количества уникальных товаров после предварительной фильтрации
n_items_after = data_train['item_id'].nunique()

# Вывод сообщения об уменьшении количества товаров после фильтрации
print('Уменьшено количество товаров с {} до {}'.format(n_items_before, n_items_after))


Decreased # items from 86865 to 5001


In [ ]:
# Создание матрицы пользователь-товар с помощью сводной таблицы (pivot table)
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id',  # Индексирование по 'user_id'
                                  columns='item_id',  # Колонки по 'item_id'
                                  values='quantity',  # Значения для заполнения матрицы (количество)
                                  aggfunc='count',  # Функция агрегирования (подсчет количества)
                                  fill_value=0  # Значение для заполнения отсутствующих элементов
                                 )

# Приведение типа данных матрицы к типу float (необходимо для модуля implicit)
user_item_matrix = user_item_matrix.astype(float)

# Вывод первых 5 строк полученной матрицы пользователь-товар
user_item_matrix.head(5)


item_id,818980,818981,819063,819112,819304,819330,819927,820082,820291,820321,...,13512965,13654702,13672065,13777104,13842090,13842107,13842160,13842224,13907063,13910669
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Получение индексов пользователей и товаров из матрицы user_item_matrix
userids = user_item_matrix.index.values  # Индексы пользователей
itemids = user_item_matrix.columns.values  # Индексы товаров

# Создание массивов индексов пользователей и товаров для новых матриц
matrix_userids = np.arange(len(userids))  # Создание массива индексов пользователей
matrix_itemids = np.arange(len(itemids))  # Создание массива индексов товаров

# Создание словарей соответствия между идентификаторами и индексами
id_to_itemid = dict(zip(matrix_itemids, itemids))  # Словарь: индекс -> идентификатор товара
id_to_userid = dict(zip(matrix_userids, userids))  # Словарь: индекс -> идентификатор пользователя

itemid_to_id = dict(zip(itemids, matrix_itemids))  # Словарь: идентификатор товара -> индекс
userid_to_id = dict(zip(userids, matrix_userids))  # Словарь: идентификатор пользователя -> индекс


In [ ]:
# Применение метода BM25 для взвешивания значений в матрице пользователь-товар
user_item_matrix = bm25_weight(user_item_matrix)

In [ ]:
%%time

# Создание модели рекомендаций на основе Alternating Least Squares (ALS)
model = AlternatingLeastSquares(factors=128,  # Количество латентных факторов
                                regularization=0.05,  # Коэффициент регуляризации
                                iterations=15,  # Количество итераций обучения
                                calculate_training_loss=True,  # Вычисление функции потерь на этапе обучения
                                num_threads=4  # Количество потоков для вычислений
                                )

# Обучение модели на преобразованной в разреженную матрицу user_item_matrix
model.fit(csr_matrix(user_item_matrix).tocsr(),  # Преобразование в разреженную матрицу формата CSR
          show_progress=True  # Вывод прогресса обучения
          )

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 2min 40s, sys: 3min 55s, total: 6min 36s
Wall time: 3min 55s


-----

In [ ]:
def get_recommendations(user, model, N=5):
    # Получение индекса пользователя из словаря userid_to_id
    user_id = userid_to_id.get(user)

    # Если пользователь не найден в словаре, возвращается список с нулём
    if user_id is None:
        return [0]

    # Получение рекомендаций для пользователя
    res = [id_to_itemid[rec] for rec in
           # Получение списка рекомендаций от модели для конкретного пользователя
           model.recommend(
               userid=userid_to_id[user],  # Индекс пользователя в модели
               user_items=csr_matrix(user_item_matrix).tocsr()[userid_to_id[user]],  # Разреженная user-item матрица
               N=N,  # Количество рекомендаций
               filter_already_liked_items=False,  # Фильтрация уже лайкнутых пользователем товаров
               filter_items=[itemid_to_id[999999]],  # Фильтрация конкретного товара (если нужно)
               recalculate_user=True  # Пересчитать профиль пользователя для улучшения рекомендаций
           )[0]]  # Обращение к первому элементу возвращаемого списка (рекомендации)

    return res  # Возвращение списка рекомендованных товаров



In [ ]:
%%time

# Для каждого пользователя в DataFrame 'result' получаем рекомендации с помощью функции 'get_recommendations' и модели 'model'.
# Результаты рекомендаций (5 элементов) добавляются в столбец 'bm25' в DataFrame 'result'.
result['bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

# Применяем функцию 'precision_at_k' к каждой строке DataFrame 'result', чтобы вычислить точность рекомендаций по K элементам.
# Затем вычисляем среднее значение точности всех пользователей.
result.apply(lambda row: precision_at_k(row['bm25'], row['actual']), axis=1).mean()


CPU times: user 100 ms, sys: 126 ms, total: 226 ms
Wall time: 126 ms


0.1265426052889324

In [14]:
# Перенесём class MainRecommender в модуль src.recommenders.py
import os

# Название директории и файла для класса
directory_name = 'src'
file_name = 'recommenders.py'

# Проверяем существование директории, и если её нет, создаем
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Код класса, который нужно поместить в файл
class_code = '''
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать

        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)

        if weighting:
          self.preserved_user_item_matrix = self.user_item_matrix
          self.user_item_matrix = bm25_weight(self.user_item_matrix)

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data):

        user_item_matrix = pd.pivot_table(data,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

        user_item_matrix = user_item_matrix.astype(float)

        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).tocsr(), show_progress=True)

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=128, regularization=0.05, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                             regularization=regularization,
                                             iterations=iterations,
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).tocsr(), show_progress=True)

        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # Получаем идентификатор пользователя из его имени
        user_id = self.userid_to_id[user]

        # Получаем топ-N купленных пользователем товаров
        top_items = self.preserved_user_item_matrix.drop(columns=999999).loc[user_id].sort_values(ascending=False).head(N)

        # Получаем похожие товары для каждого из топ-N товаров
        similar_items = []
        for item_id, score in top_items.items():
            recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)
            similar_items.append(self.id_to_itemid[recs[0][1]])

        # Удаляем дубликаты и получаем список рекомендаций товаров, похожих на топ-N купленных пользователем товаров
        recommended_items = list(set(similar_items))

        assert len(recommended_items) == N, 'Количество рекомендаций != {}'.format(N)
        return recommended_items

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        similar_users = self.model.similar_users(self.userid_to_id[user], N=6)

        similar_user_items = []

        for sim_user_id in similar_users[0][1:]:

          recs = self.own_recommender.recommend(userid= sim_user_id,
                          user_items=csr_matrix(self.user_item_matrix).tocsr()[sim_user_id],   # на вход user-item matrix
                          N=5,
                          filter_already_liked_items=False,
                          filter_items=None,
                          recalculate_user=False)
          similar_user_items.append(self.id_to_itemid[recs[0][0]])

        # Удаляем дубликаты и получаем список рекомендаций товаров, похожих на топ-N купленных пользователем товаров
        res = list(set(similar_user_items))

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
'''

# Путь к файлу .py
file_path = os.path.join(directory_name, file_name)

# Создаем файл и записываем код класса в него
with open(file_path, 'w') as file:
    file.write(class_code)

----

Проверка, что все работает

In [ ]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender



In [2]:
# Чтение данных из файла CSV и сохранение в переменную 'data'
data = pd.read_csv('/content/drive/MyDrive/lesson_4/lesson_4/homework/retail_train.csv')

# Приведение названий столбцов к нижнему регистру для удобства работы с данными
data.columns = [col.lower() for col in data.columns]

# Переименование столбцов 'household_key' на 'user_id' и 'product_id' на 'item_id' для улучшения читаемости данных
data.rename(columns={'household_key': 'user_id', 'product_id': 'item_id'}, inplace=True)

# Определение количества недель для тестирования
test_size_weeks = 3

# Создание датасета для тренировки (данные до определенного номера недели)
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]

# Создание датасета для тестирования (данные после определенного номера недели)
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Вывод первых 5 строк тренировочного датасета для предварительного анализа
data_train.head(5)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
# Группировка данных по 'item_id' и суммирование количества проданных товаров ('quantity') для каждого товара
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()

# Переименование столбца 'quantity' на 'n_sold' для более ясного понимания
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

# Сортировка товаров по количеству проданных единиц ('n_sold') в порядке убывания и выбор топ-5000 самых популярных товаров
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Если товар не входит в топ-5000 по популярности, заменить 'item_id' на 999999
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999


In [8]:
# Создаем экземпляр класса MainRecommender с данными user_item_data
recommender = MainRecommender(data_train)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [13]:
# Получаем рекомендации для конкретного пользователя
user_id = 10  # Идентификатор пользователя, для которого запрашиваются рекомендации

# Получение рекомендаций похожих товаров для указанного пользователя (user_id)
similar_items = recommender.get_similar_items_recommendation(user_id, N=5)

# Получение рекомендаций похожих пользователей для указанного пользователя (user_id)
similar_users = recommender.get_similar_users_recommendation(user_id, N=5)



In [15]:
# Вывод
similar_items

[1060005, 988208, 10457112, 1102845, 948254]

In [16]:
similar_users

[1101010, 991379, 908314, 852317, 1108094]